Скрипт разбирает Реестр лицензий РосЗдравнадзора в плоский формат из XML.

От себя добавляет количество адресов (помещений, на которые оформляется лицензия) и нумерацию этих адресов.

Исходный датасет: https://roszdravnadzor.gov.ru/opendata/7710537160-licenses

### Описание столбцов ###

столбец — описание РЗН (на канцелярском) / перевод на русский

**name** — Наименование лицензирующего органа / Кто выдал лицензию

**activity_type** — Лицензируемый вид деятельности / На что выдали

**full_name_licensee** — Полное наименование лицензиата / Название компании

**abbreviated_name_licensee** — Сокращенное наименование лицензиата

**brand_name_licensee** — Фирменное наименование лицензиата

**form** — Организационно-правовая форма

**address_region** — Регион места нахождения юридического лица либо регион места жительства индивидуального предпринимателя

**address** — Адрес места нахождения юридического лица либо адрес места жительства индивидуального предпринимателя

**ogrn** —	ОГРН/ОГРНИП лицензиата

**inn** — ИНН лицензиата

**address_number** — none / Количество адресов (помещений, отделений, филиалов и т.п.), на которые выдана лицензия

**address_nn** — none / Номер адреса

**sub_address** — Адреса мест осуществления лицензируемого вида деятельности

**sub_index** —	Индекс

**sub_region** — Регион

**sub_city** —	Населенный пункт

**sub_street** — Улица, дом, офис и т.д

**sub_code_fias** —	Код ФИАС

**work_types** — Выполняемые работы, оказываемые услуги

**number** — Номер лицензии

**date** — Дата выдачи лицензии

**number_orders** — Номер приказа (распоряжения) о предоставлении лицензии

**date_order** — Дата приказа (распоряжения) о предоставлении лицензии

**date_register** — Дата внесения записи в реестр

**number_duplicate** —	Номер дубликата лицензии

**date_duplicate** — Дата выдачи дубликата лицензии

**termination** — Основание прекращения действия лицензии

**date_termination** — Дата прекращения действия лицензии

**information** — Сведения о проведенных проверках

**information_regulations** — Сведения о выданных постановлениях по результатам проверок

**information_suspension_resumption** — Сведения о приостановлении/возобновлении действия лицензии

**information_cancellation** — Сведения об аннулировании лицензии

**information_reissuing** — Сведения о переоформлении лицензии

**date_reissuing** — Дата переоформления лицензии

In [1]:
import xml.etree.ElementTree as ET

import pandas as pd

In [2]:
# путь к файлу

file_path = 'C:/00_Data/datasets/RosZdravNadzor/'

In [5]:


xml_data = open(file_path + 'reestr_RZN_2021_11_28.xml', 'r', encoding='utf-8').read()

In [ ]:
root = ET.XML(xml_data)  # Parse XML

In [3]:
xml_data

NameError: name 'xml_data' is not defined

In [ ]:
%%time

data = []

for element in root:
    name = element.find('name').text
    activity_type = element.find('activity_type').text
    full_name_licensee = element.find('full_name_licensee').text
    abbreviated_name_licensee = element.find('abbreviated_name_licensee').text
    brand_name_licensee = element.find('brand_name_licensee').text
    form = element.find('form').text
    address_region = element.find('address_region').text
    address = element.find('address').text
    ogrn = element.find('ogrn').text
    inn = element.find('inn').text
    address_number = len(element.find('work_address_list'))
    number = element.find('number').text
    date = element.find('date').text
    number_orders = element.find('number_orders').text
    date_order = element.find('date_order').text
    date_register = element.find('date_register').text
    number_duplicate = element.find('number_duplicate').text
    date_duplicate = element.find('date_duplicate').text
    termination = element.find('termination').text
    date_termination = element.find('date_termination').text
    information = element.find('information').text
    information_regulations = element.find('information_regulations').text
    information_suspension_resumption = element.find('information_suspension_resumption').text
    information_cancellation = element.find('information_cancellation').text
    information_reissuing = element.find('information_reissuing').text
    date_reissuing = element.find('date_reissuing').text
    
    for i in range(address_number): # цикл для разбора данных филиалов
        work_types = []
        address_nn = i + 1
        sub_address = element[10][i].find('address').text
        sub_region = element[10][i].find('region').text
        sub_index = element[10][i].find('index').text
        sub_city = element[10][i].find('city').text
        sub_street = element[10][i].find('street').text
        sub_code_fias = element[10][i].find('code_fias').text
        
        
        for work_type in element[10][i][6]: # цикл для сбора видов деятельности
            work_types += [work_type.text]  
            
            
        # добавляю данные в список, в теле 2-го цикла, чтобы "распластать" данные
        data.append([name, activity_type,
                     full_name_licensee, abbreviated_name_licensee, brand_name_licensee,
                     form, address_region, address,
                     ogrn, inn,
                     address_number, address_nn,
                     sub_address,
                     sub_index, sub_region, sub_city, sub_street,
                     sub_code_fias,
                     work_types,
                     number, date, number_orders, date_order, date_register,
                     number_duplicate, date_duplicate,
                     termination, date_termination,
                     information, information_regulations,
                     information_suspension_resumption,
                     information_cancellation, information_reissuing, date_reissuing])

In [ ]:
# создаю датафрейм на основе списка
head_list = ['name', 'activity_type',
             'full_name_licensee', 'abbreviated_name_licensee', 'brand_name_licensee',
             'form', 'address_region', 'address',
             'ogrn', 'inn',
             'address_number', 'address_nn',
             'sub_address',
             'sub_index', 'sub_region', 'sub_city', 'sub_street',
             'sub_code_fias',
             'work_types',
             'number', 'date', 'number_orders', 'date_order', 'date_register',
             'number_duplicate', 'date_duplicate',
             'termination', 'date_termination',
             'information', 'information_regulations',
             'information_suspension_resumption',             'information_cancellation', 'information_reissuing', 'date_reissuing']

work = pd.DataFrame(data, columns=head_list)

In [ ]:
work.shape

In [ ]:
# записываю в csv

work.to_csv('C:/00_Data/РосЗдравНадзор/RZN_lic_2021_11_28.csv', encoding='utf-8')

In [ ]:
# записываю в файл

work.to_excel('C:/00_Data/РосЗдравНадзор/RosZdravNadzor/RZN_lic_2021_10_24.xlsx', index=False)